## Homework 1
#### DS7337: Natural Language Processing, Fall 2022
By: Allen Hoskins

***
## 1. Install Python and NLTK

In [9]:
!python -V

Python 3.9.13


In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Image
from IPython.core.display import HTML 
import requests
import urllib.request
import time
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

In [ ]:
#### Method to get the Vocab Size and normalize the score

def n_vocab_size(*arg):
    vocab_size = np.array([])
    vocab_size_norm = np.array([])
    
    #### Getting the Vocab Size
    for text in arg:
        vocab_size = np.append(vocab_size,len(set(text)))

    #### Normalizing using sklearn preprocessing 
    vocab_size_norm = MinMaxScaler(vocab_size, feature_range=(0,1), axis=0)
    
    return(vocab_size,vocab_size_norm)



In [ ]:
vocab_size = n_vocab_size(text1,text2,text3,text4,text5,text6,text7,text8,text9)

In [ ]:
print("Normalized Values by using sklearn", *vocab_size[2],sep='\n- ')